In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import numpy as np

In [2]:
cancer = load_breast_cancer()
x = cancer.data
y = cancer.target
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

In [11]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify=y_train, test_size=0.2, random_state=42)
print(len(x_train), len(x_val))

364 91


In [12]:
train_mean = np.mean(x_train, axis=0) # 평균 계산, axis를 0으로 지정하면 2차원 배열 열을 기준으로 통계치를 계산하여 하나의 행 벡터로 반환
train_std = np.std(x_train, axis=0) # 표준편차 계산
x_train_scaled = (x_train - train_mean) / train_std # train set 표준화 : (특성값 - 평균)/표준편차

In [13]:
val_mean = np.mean(x_val, axis=0)
val_std = np.std(x_val, axis=0)
x_val_scaled = (x_val - val_mean) / val_std

In [14]:
sgd = SGDClassifier(loss='log', penalty='l2', alpha=0.001, random_state=42)
sgd.fit(x_train_scaled, y_train)
sgd.score(x_val_scaled, y_val)

0.978021978021978

# * Cross validation

In [6]:
from sklearn.model_selection import cross_validate
sgd = SGDClassifier(loss='log', penalty='l2', alpha=0.001, random_state=42)
scores = cross_validate(sgd, x_train, y_train, cv=10)
print(np.mean(scores['test_score']))

0.850096618357488


표준화 처리를 하지 않았기 때문에 낮음.

훈련 세트 전체를 전처리 한 후 cross_validation() 함수에 전달하면 검증 폴드 누설

# * Pipeline 클래스 사용하여 교차 검증 수행

In [7]:
from sklearn.pipeline import make_pipeline # 파이프라인 객체 생성 클래스 임포트
from sklearn.preprocessing import StandardScaler # 표준화 전처리 수행하는 클래스 임포트

pipe = make_pipeline(StandardScaler(), sgd) # 전처리 단계와 SGDClassifier 클래스를 하나로 연결해주는 Pipeline 클래스 생성
scores = cross_validate(pipe, x_train, y_train, cv=10, return_train_score=True)
# return_train_score 매개변수를 True로 설정하면 훈련 폴드의 점수도 얻을 수 있음
print(np.mean(scores['test_score']))

0.9694202898550724


In [8]:
print(np.mean(scores['train_score']))

0.9875478561631581
